<a href="https://colab.research.google.com/github/BritCrit/monthly_automotive_incentives_autoblog/blob/main/car_incentives_autoblog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

url = 'https://www.autoblog.com/new-car-incentives/'

def collect_incentives(url):
  return pd.read_html(url)

def find_page_count(df):
  page_count = df[0][-1:].iloc[0][0]
  return [int(s) for s in page_count.split() if s.isdigit()][0]\

def all_incenvtives(url):
  df_frames = []
  base_df = collect_incentives(url)
  df_frames.append(base_df[0])
  page_count = find_page_count(base_df)
  print(page_count)
  for n in range(2 , page_count):
    tmp_df = collect_incentives(f"https://www.autoblog.com/new-car-incentives/pg-{n}/")
    print(tmp_df)
    if tmp_df != None:
      df_frames.append(tmp_df[0])
    else: 
      pass
  return df_frames

def combine_dataframes(url):
  collected_dataframes = all_incenvtives(url)
  return pd.concat(collected_dataframes, ignore_index=True).ffill(axis=0)

master_df = combine_dataframes(url)

#Update columns 
master_df.columns = ['model', 'incentive', 'brand']
#Drop repeating lines and page breaks
master_df = master_df[master_df['incentive'] != master_df['brand']]
#Remove unicode formatting 
master_df.incentive = master_df.incentive.apply(lambda x: x.replace(u'\xa0', ' '))

master_df.to_csv("incentives.csv", index=False)